In [1]:
#라이브러리 임포트
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import BatchNormalization
from keras.layers import Dropout
from keras.layers import Activation
from keras import regularizers
import numpy as np

#이미지 생성용
from keras.preprocessing.image import ImageDataGenerator
from collections import Counter

# 콜백 함수
from keras.callbacks import TensorBoard
from keras.callbacks import EarlyStopping

#시간 구하기용
from datetime import datetime

Using TensorFlow backend.


In [2]:
def ModelDesign(size_x, size_y, depth, label_num, regularizer_value):     
    #초기화
    model = Sequential() 
    
    # CNN 레이어
    # 레이어1(Input)
    model.add(Conv2D(filters=96,
                     input_shape=(size_x,size_y,depth),
                     kernel_size=(11,11),
                     strides=(4,4),
                     padding='valid',
                     kernel_regularizer=regularizers.l2(regularizer_value)))
    # Activation
    model.add(Activation('relu'))
    # Pooling 
    model.add(MaxPooling2D((2,2), strides=(1,1), padding='valid'))
    # Batch Normalisation before passing it to the next layer
    model.add(BatchNormalization())
    
    # 레이어2
    model.add(Conv2D(filters=256, kernel_size=(11,11), 
                     strides=(1,1), padding='valid', 
                     kernel_regularizer=regularizers.l2(regularizer_value)))
    model.add(Activation('relu'))
    # Pooling
    model.add(MaxPooling2D((2,2), strides=(1,1), padding='valid'))
    # Batch Normalisation
    model.add(BatchNormalization())
    
    # 레이어3
    model.add(Conv2D(filters=384, kernel_size=(3,3), 
                     strides=(1,1), padding='valid', 
                     kernel_regularizer=regularizers.l2(regularizer_value)))
    model.add(Activation('relu'))
    # Batch Normalisation
    model.add(BatchNormalization())
    
    # 레이어4
    model.add(Conv2D(filters=384, kernel_size=(3,3), 
                     strides=(1,1), padding='valid', 
                     kernel_regularizer=regularizers.l2(regularizer_value)))
    model.add(Activation('relu'))
    # Batch Normalisation
    model.add(BatchNormalization())
    
    # 레이어5
    model.add(Conv2D(filters=256, kernel_size=(3,3), 
                     strides=(1,1), padding='valid', 
                     kernel_regularizer=regularizers.l2(regularizer_value)))
    model.add(Activation('relu'))
    # Pooling
    model.add(MaxPooling2D((2,2), strides=(2,2), padding='valid'))
    # Batch Normalisation
    model.add(BatchNormalization())
    
    # Dense 레이어
    # Flatten
    model.add(Flatten())
    
    # 레이어1(Input2)
    model.add(Dense(4096, input_shape=(size_x*size_y*depth,), 
                    kernel_regularizer=regularizers.l2(regularizer_value)))
    model.add(Activation('relu'))
    # Dropout
    model.add(Dropout(0.4))
    # Batch Normalisation
    model.add(BatchNormalization())

    # 레이어2
    model.add(Dense(4096, kernel_regularizer=regularizers.l2(regularizer_value)))
    model.add(Activation('relu'))
    # Dropout
    model.add(Dropout(0.4))
    # Batch Normalisation
    model.add(BatchNormalization())

    # 레이어3
    model.add(Dense(1000, kernel_regularizer=regularizers.l2(regularizer_value)))
    model.add(Activation('relu'))
    # Dropout
    model.add(Dropout(0.4))
    # Batch Normalisation
    model.add(BatchNormalization())
    
    # 출력 레이어
    model.add(Dense(label_num))
    model.add(Activation('softmax'))
    
    #모델 확인
    model.summary()
    
    return model

In [4]:
# 설정값들
IMAGE_SIZE = 128
IMAGE_DEPTH = 3
LABEL_NUM = 16
REGULIZER = 0.0001

#GetCNN Model
model = ModelDesign(IMAGE_SIZE, IMAGE_SIZE, IMAGE_DEPTH, LABEL_NUM, REGULIZER)

#Compling CNN
#model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 30, 30, 96)        34944     
_________________________________________________________________
activation_10 (Activation)   (None, 30, 30, 96)        0         
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 29, 29, 96)        0         
_________________________________________________________________
batch_normalization_9 (Batch (None, 29, 29, 96)        384       
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 19, 19, 256)       2973952   
_________________________________________________________________
activation_11 (Activation)   (None, 19, 19, 256)       0         
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 18, 18, 256)       0         
__________

In [4]:
#배치 크기
BATCH_SIZE = 16

#훈련용 이미지 생성
train_datagen = ImageDataGenerator(horizontal_flip = True)
validation_datagen = ImageDataGenerator(horizontal_flip = True)

training_set = train_datagen.flow_from_directory('./Dataset/Train',
                                                target_size = (IMAGE_SIZE,IMAGE_SIZE),
                                                batch_size = BATCH_SIZE)

validation_set = validation_datagen.flow_from_directory('./Dataset/Validation',
                                           target_size = (IMAGE_SIZE,IMAGE_SIZE),
                                           batch_size = BATCH_SIZE)

# Val값에 따라 훈련 조기종료 설정
early_stopping = EarlyStopping()

# Class Weights 계산
# 가장 많은 클레스의 수 / 각 클레스의 수
# 클레스 = 각 라벨에 해당하는 데이터 수
counter = Counter(training_set.classes)
max_val = float(max(counter.values()))
class_weights = {class_id : max_val/num_images for class_id, num_images in counter.items()}

#배치 수 계산
train_step_epoch = int(len(training_set.classes) / BATCH_SIZE)
validation_step_epoch = int(len(validation_set.classes) / BATCH_SIZE)

Found 69267 images belonging to 16 classes.
Found 9588 images belonging to 16 classes.


In [5]:
#오늘 날짜_시간
NOW = datetime.now().strftime('%Y%m%d_%H_%M_%S')

#텐서보드 로그
tensorboard = TensorBoard(log_dir="./Logs/AlexNet/" + NOW)

#훈련!
model.fit_generator(training_set,
                    steps_per_epoch = train_step_epoch,
                    epochs = 150,
                    validation_data = validation_set,
                    validation_steps = validation_step_epoch,
                    #class_weight=class_weights,
                    shuffle=True,
                    callbacks=[tensorboard])

#모델 저장
model_json = model.to_json()
with open("./Models/AlexNet/model_alex_" + NOW + ".json", "w") as json_file : 
    json_file.write(model_json)

# Weight 저장
model.save_weights("./Models/AlexNet/model_alex_weight_" + NOW + ".h5")
print("Saved model to disk")

Epoch 1/150
4329/4329 [==============================] - 1054s 243ms/step - loss: 3.8040 - acc: 0.2080 - val_loss: 3.6037 - val_acc: 0.2356
Epoch 2/150
4329/4329 [==============================] - 345s 80ms/step - loss: 3.5113 - acc: 0.2409 - val_loss: 3.5635 - val_acc: 0.2353
Epoch 3/150
4329/4329 [==============================] - 345s 80ms/step - loss: 3.3797 - acc: 0.2575 - val_loss: 3.3805 - val_acc: 0.2351
Epoch 4/150
4329/4329 [==============================] - 345s 80ms/step - loss: 3.1970 - acc: 0.2627 - val_loss: 3.1672 - val_acc: 0.2362
Epoch 5/150
4329/4329 [==============================] - 345s 80ms/step - loss: 3.0004 - acc: 0.2623 - val_loss: 3.1488 - val_acc: 0.2355
Epoch 6/150
4329/4329 [==============================] - 345s 80ms/step - loss: 2.9581 - acc: 0.2626 - val_loss: 3.1164 - val_acc: 0.2355
Epoch 7/150
4329/4329 [==============================] - 345s 80ms/step - loss: 2.9624 - acc: 0.2625 - val_loss: 3.1021 - val_acc: 0.2350
Epoch 8/150
4329/4329 [=========

Epoch 60/150
4329/4329 [==============================] - 345s 80ms/step - loss: 2.6088 - acc: 0.2655 - val_loss: 2.8651 - val_acc: 0.2372
Epoch 61/150
4329/4329 [==============================] - 345s 80ms/step - loss: 2.6000 - acc: 0.2658 - val_loss: 3.1900 - val_acc: 0.2093
Epoch 62/150
4329/4329 [==============================] - 345s 80ms/step - loss: 2.6105 - acc: 0.2665 - val_loss: 3.0661 - val_acc: 0.2230
Epoch 63/150
4329/4329 [==============================] - 345s 80ms/step - loss: 2.6253 - acc: 0.2660 - val_loss: 3.2491 - val_acc: 0.2396
Epoch 64/150
4329/4329 [==============================] - 345s 80ms/step - loss: 2.6119 - acc: 0.2655 - val_loss: 3.1170 - val_acc: 0.2288
Epoch 65/150
4329/4329 [==============================] - 345s 80ms/step - loss: 2.6056 - acc: 0.2665 - val_loss: 5.2671 - val_acc: 0.1966
Epoch 66/150
4329/4329 [==============================] - 345s 80ms/step - loss: 2.6081 - acc: 0.2665 - val_loss: 2.8506 - val_acc: 0.2368
Epoch 67/150
4329/4329 [===

In [35]:
#테스트셋 불러오기
test_datagen = ImageDataGenerator(horizontal_flip = True)
test_set = validation_datagen.flow_from_directory('./Dataset/Test',
                                           target_size = (IMAGE_SIZE,IMAGE_SIZE),
                                           batch_size = BATCH_SIZE)

# 모델 평가하기
print("-- Evaluate --")
scores = model.evaluate_generator(
            test_set, 
            steps = 100)

print("%s: %.2f%%" %(model.metrics_names[1], scores[1]*100))

# 모델 예측하기
print("-- Predict --")
output = model.predict_generator(test_set, steps=5)
np.set_printoptions(formatter={'float': lambda x: "{0:0.3f}".format(x)})
print(test_set.class_indices)
print(output[0])

-- Evaluate --
acc: 22.72%
-- Predict --
{'A': 0, 'AB': 1, 'ABC': 2, 'ABCD': 3, 'ABD': 4, 'AC': 5, 'ACD': 6, 'AD': 7, 'B': 8, 'BC': 9, 'BCD': 10, 'BD': 11, 'C': 12, 'CD': 13, 'D': 14, 'N': 15}
[0.001 0.000 0.000 0.000 0.000 0.000 0.000 0.075 0.000 0.047 0.000 0.000
 0.000 0.002 0.002 0.871]


In [ ]:
# 모델 평가하기 - 트레이닝 셋으로
print("-- Evaluate --")
scores = model.evaluate_generator(
            training_set, 
            steps = 100)

print("%s: %.2f%%" %(model.metrics_names[1], scores[1]*100))

# 모델 예측하기 - 트레이닝 셋으로
print("-- Predict --")
output = model.predict_generator(training_set, steps=5)
np.set_printoptions(formatter={'float': lambda x: "{0:0.3f}".format(x)})
print(training_set.class_indices)
print(output[0])